#Data Collection
## Which schools have solar panels already? 
Data from opendata.dk

# API for querying "aktuel"
## Query example (first 5 results):

https://admin.opendata.dk/api/3/action/datastore_search?resource_id=251528ca-8ec9-4b70-9960-83c4d0c4e7b6 &limit=5

## Query example (results containing "jones"):

https://admin.opendata.dk/api/3/action/datastore_search?q=jones&resource_id=251528ca-8ec9-4b70-9960-83c4d0c4e7b6

## Query example (via SQL statement):

https://admin.opendata.dk/api/3/action/datastore_search_sql?sql=SELECT * from "251528ca-8ec9-4b70-9960-83c4d0c4e7b6" WHERE title LIKE 'jones'


# Querying API for produktion

## Query example (first 5 results):

https://admin.opendata.dk/api/3/action/datastore_search?resource_id=3028c112-a089-474d-a9c1-73d1c8352fca &limit=5

## Query example (results containing "jones"):

https://admin.opendata.dk/api/3/action/datastore_search?q=jones&resource_id=3028c112-a089-474d-a9c1-73d1c8352fca

## Query example (via SQL statement):

https://admin.opendata.dk/api/3/action/datastore_search_sql?sql=SELECT * from "3028c112-a089-474d-a9c1-73d1c8352fca" WHERE title LIKE 'jones'

In [ ]:
pip install folium

In [ ]:
import folium

In [1]:
import pandas as pd

In [2]:
sample = pd.read_csv("../data/raw/aktuel.csv")

In [3]:
schools = pd.read_csv("../data/raw/skoler.csv")

In [4]:
panels = pd.read_csv("../data/raw/metadata.csv")

In [5]:
schools.drop(columns=['_id', 'Periode', 'Adresse', 'Postnr', 'Skolekode', 'Antal', 'LINK', 'Ledernavn', 'Ledermail', 'Opdateret'], inplace=True)


In [6]:
schools.head()

,Skolenavn,lat,lng,Postdist
0,Stensagerskolen,56.166771,10.137611,Brabrand
1,Bakkegårdsskolen,56.253349,10.148964,Trige
2,Ellevangskolen,56.203037,10.217692,Risskov
3,Beder Skole,56.061577,10.211610,Beder
4,Elev Skole,56.241188,10.199622,Lystrup


In [7]:
panels.drop(columns=['_id', 'date', 'magistrat'], inplace=True)


In [10]:
# Merge the dataframes to add a column indicating the presence of solar panels
merged_df = schools.merge(panels, left_on='Skolenavn', right_on='location', how='left', indicator=True)

# Replace NaN values with 0 in the 'sid' column and convert it to integer
merged_df['sid'] = merged_df['sid'].fillna(0).astype(int)

# Add a new column 'Has Solar' based on the merge indicator
merged_df['Has Solar'] = merged_df['_merge'].apply(lambda x: 'Has solar' if x == 'both' else "Doesn't have solar")

# Drop the merge indicator column
merged_df = merged_df.drop(columns=['_merge', 'location'])

# Separate the dataframes
schools_with_solar = merged_df[merged_df['Has Solar'] == 'Has solar']
schools_without_solar = merged_df[merged_df['Has Solar'] == "Doesn't have solar"]

# Optional: Reset index for the resulting dataframes
schools_with_solar.reset_index(drop=True, inplace=True)
schools_without_solar.reset_index(drop=True, inplace=True)


In [12]:
merged_df.head()

,Skolenavn,lat,lng,Postdist,sid,Has Solar
0,Stensagerskolen,56.166771,10.137611,Brabrand,0,Doesn't have solar
1,Bakkegårdsskolen,56.253349,10.148964,Trige,22224,Has solar
2,Ellevangskolen,56.203037,10.217692,Risskov,0,Doesn't have solar
3,Beder Skole,56.061577,10.211610,Beder,0,Doesn't have solar
4,Elev Skole,56.241188,10.199622,Lystrup,0,Doesn't have solar


In [25]:
import folium
from folium.plugins import MarkerCluster
from folium import IFrame, Popup

# Initialize the map with a greyscale tile
m = folium.Map(location=[schools['lat'].mean(), schools['lng'].mean()], zoom_start=10, tiles='cartodbpositron')

# Add title
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Aarhus Folkeskoler Solar Panel Map</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Function to choose marker icon
def get_marker_icon(has_solar):
    icon_color = 'orange' if has_solar == 'Has solar' else 'darkred'
    icon = 'sun' if has_solar == 'Has solar' else 'times'
    return folium.Icon(color=icon_color, icon=icon, prefix='fa')  # 'fa' prefix for FontAwesome icons

# Add markers to the map
for idx, row in merged_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        icon=get_marker_icon(row['Has Solar']),
        tooltip=f"School: {row['Skolenavn']}<br>Postal District: {row['Postdist']}<br>SID: {row['sid']}"
    ).add_to(m)

# Save the map to an HTML file
m.save('schools_map.html')


import folium

# Initialize the map with a greyscale tile
m = folium.Map(location=[schools['lat'].mean(), schools['lng'].mean()], zoom_start=10, tiles='cartodbpositron')

# Function to choose marker icon
def get_marker_icon(has_solar):
    icon_color = 'orange' if has_solar == 'Has solar' else 'gray'
    icon = 'sun' if has_solar == 'Has solar' else 'times'
    return folium.Icon(color=icon_color, icon=icon, prefix='fa')  # 'fa' prefix for FontAwesome icons

# Add markers to the map
for idx, row in merged_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        icon=get_marker_icon(row['Has Solar']),
        tooltip=f"School: {row['Skolenavn']}<br>Postal District: {row['Postdist']}<br>SID: {row['sid']}"
    ).add_to(m)

# Save the map to an HTML file
m.save('../results/schools_map.html')


In [26]:
m